# Lab | Cross Validation

For this lab, we will build a model on customer churn binary classification problem. You will be using `files_for_lab/Customer-Churn.csv` file.



### Instructions

1. Apply SMOTE for upsampling the data

    - Use logistic regression to fit the model and compute the accuracy of the model.
    - Use decision tree classifier to fit the model and compute the accuracy of the model.
    - Compare the accuracies of the two models.


2. Apply TomekLinks for downsampling

    - It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.
    - Use logistic regression to fit the model and compute the accuracy of the model.
    - Use decision tree classifier to fit the model and compute the accuracy of the model.
    - Compare the accuracies of the two models.
    - You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.tree import DecisionTreeClassifier

In [2]:
churn_data = pd.read_csv('W7 - D3 - Customer-Churn.csv')
churn_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
churn_data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
churn_data['TotalCharges'] = pd.to_numeric(churn_data['TotalCharges'], errors='coerce')

In [5]:
data = churn_data.dropna(axis='index')

In [6]:
data.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
num_data = data.select_dtypes(include=['int64', 'float64'])
num_data.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65


In [8]:
cat_data = data.select_dtypes(include=['object'])
cat_data = cat_data.drop(['Churn'], axis=1)
cat_data.head()

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month


In [9]:
dummy_data = pd.get_dummies(cat_data,drop_first=True)
dummy_data.head()

,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year
0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
data['Churn'].value_counts()

No     5163
Yes    1869
Name: Churn, dtype: int64

### 1. SMOTE

In [11]:
data_sm = pd.concat([num_data,data['Churn'],dummy_data],axis=1)
smote = SMOTE()
y = data_sm['Churn']
X = data_sm.drop(['Churn'], axis=1)
X_sm, y_sm = smote.fit_sample(X, y)

unique, counts = np.unique(y_sm, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 5163]
 ['Yes' 5163]]


In [12]:
transformer = StandardScaler().fit(X_sm)
X_sm_standarized = transformer.transform(X_sm)
X_sm_standarized = pd.DataFrame(X_sm_standarized)
X_sm_standarized.columns = X_sm_standarized.columns.astype(str)
X_sm_standarized.rename(columns={'0':'SeniorCitizen', '1':'tenure', '2':'MonthlyCharges', '3':'TotalCharges'}, inplace=True)
X_sm_standarized.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.397835,-1.114378,-1.336909,-0.925096,-0.868180,1.285478,-0.524978,-2.961276,-0.43861,-0.511445,...,-0.43861,-0.614289,-0.43861,-0.515059,-0.43861,-0.743179,-0.43861,-0.750732,-0.41994,-0.445497
1,-0.397835,0.261018,-0.390000,-0.074139,1.151835,-0.777921,-0.524978,0.337692,-0.43861,1.955244,...,-0.43861,1.627898,-0.43861,-0.515059,-0.43861,-0.743179,-0.43861,-0.750732,2.38129,-0.445497
2,-0.397835,-1.072699,-0.498318,-0.889266,1.151835,-0.777921,-0.524978,0.337692,-0.43861,1.955244,...,-0.43861,-0.614289,-0.43861,-0.515059,-0.43861,-0.743179,-0.43861,-0.750732,-0.41994,-0.445497
3,-0.397835,0.719484,-0.901890,-0.096447,1.151835,-0.777921,-0.524978,-2.961276,-0.43861,1.955244,...,-0.43861,1.627898,-0.43861,1.941524,-0.43861,-0.743179,-0.43861,-0.750732,2.38129,-0.445497
4,-0.397835,-1.072699,0.090443,-0.869361,-0.868180,-0.777921,-0.524978,0.337692,-0.43861,-0.511445,...,-0.43861,-0.614289,-0.43861,-0.515059,-0.43861,-0.743179,-0.43861,-0.750732,-0.41994,-0.445497


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_sm_standarized, y_sm, test_size=0.3, random_state=100)

#### 1.1. Use logistic regression to fit the model and compute the accuracy of the model.

In [14]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)

In [15]:
predictions_lm = classification.predict(X_test)

In [16]:
print("The accuracy of the model on test set is: %4.2f " % accuracy_score(y_test, predictions_lm))

The accuracy of the model on test set is: 0.83 


#### 1.2. Use decision tree classifier to fit the model and compute the accuracy of the model.

In [17]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions_dt = model.predict(X_test)
print("The accuracy of the model is: {:4.2f}".format(model.score(X_test, y_test)))

The accuracy of the model is: 0.78


#### 1.3. Compare the accuracies of the two models.

### 2. TomekLink's

In [18]:
tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_sample(X, y)

unique, counts = np.unique(y_tl, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 4595]
 ['Yes' 1869]]


C:\Users\yelam\anaconda3\lib\site-packages\imblearn\utils\_validation.py:635: FutureWarning: Pass sampling_strategy=majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


In [19]:
transformer = StandardScaler().fit(X_tl)
X_tl_standarized = transformer.transform(X_tl)
X_tl_standarized = pd.DataFrame(X_tl_standarized)
X_tl_standarized.columns = X_tl_standarized.columns.astype(str)
X_tl_standarized.rename(columns={'0':'SeniorCitizen', '1':'tenure', '2':'MonthlyCharges', '3':'TotalCharges'}, inplace=True)
X_tl_standarized.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.43738,0.035799,-0.233229,-0.181789,0.989841,-0.973133,-0.658707,0.328037,-0.545409,1.581653,...,-0.545409,1.385973,-0.545409,-0.636327,-0.545409,-0.781258,-0.545409,-0.791305,1.934549,-0.578779
1,-0.43738,-1.259441,-0.335380,-0.962402,0.989841,-0.973133,-0.658707,0.328037,-0.545409,1.581653,...,-0.545409,-0.721515,-0.545409,-0.636327,-0.545409,-0.781258,-0.545409,-0.791305,-0.516916,-0.578779
2,-0.43738,0.481037,-0.715975,-0.203152,0.989841,-0.973133,-0.658707,-3.048441,-0.545409,1.581653,...,-0.545409,1.385973,-0.545409,1.571519,-0.545409,-0.781258,-0.545409,-0.791305,1.934549,-0.578779
3,-0.43738,-1.259441,0.219860,-0.943340,-1.010263,-0.973133,-0.658707,0.328037,-0.545409,-0.632250,...,-0.545409,-0.721515,-0.545409,-0.636327,-0.545409,-0.781258,-0.545409,-0.791305,-0.516916,-0.578779
4,-0.43738,-1.016583,1.173819,-0.650240,-1.010263,-0.973133,-0.658707,0.328037,-0.545409,-0.632250,...,-0.545409,1.385973,-0.545409,-0.636327,-0.545409,1.279987,-0.545409,1.263735,-0.516916,-0.578779


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_tl_standarized, y_tl, test_size=0.3, random_state=100)

#### 2.1. It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.

#### 2.2. Use logistic regression to fit the model and compute the accuracy of the model.

In [21]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='ovr').fit(X_train, y_train)

In [22]:
predictions_lm = classification.predict(X_test)

In [23]:
print("The accuracy of the model on test set is: %4.2f " % accuracy_score(y_test, predictions_lm))

The accuracy of the model on test set is: 0.81 


#### 2.3. Use decision tree classifier to fit the model and compute the accuracy of the model.

In [24]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions_dt = model.predict(X_test)
print("The accuracy of the model is: {:4.2f}".format(model.score(X_test, y_test)))

The accuracy of the model is: 0.75


#### 2.4. Compare the accuracies of the two models.

#### 2.5. You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.

In [25]:
X_tl2, y_tl2 = tl.fit_sample(X_tl, y_tl)

unique, counts = np.unique(y_tl2, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 4429]
 ['Yes' 1869]]
